# Imports

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data

In [30]:
# Setting up GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [31]:
def make_deterministic(random_seed = 6):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)

make_deterministic()

# Graph Creation

In [32]:
node = pd.read_csv('../node_information.csv', header=None)
train = pd.read_csv('../train.txt', header=None, names=['Source','Target','Edge'], delim_whitespace=True)
test = pd.read_csv('../test.txt', header=None, names=['Source','Target'], delim_whitespace=True)

In [33]:
node.loc[:, node.columns != node.columns[0]] = node.loc[:, node.columns != node.columns[0]].astype(int)
node.head()

,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
nodes = [i for i in node[0]]
nodes[:10]

[0, 4, 5, 6, 7, 9, 10, 11, 13, 17]

In [35]:
node_idx = pd.DataFrame({'old_index': nodes, 'new_index': range(3597)})
node_idx

,old_index,new_index
0,0,0
1,4,1
2,5,2
3,6,3
4,7,4
...,...,...
3592,7584,3592
3593,7589,3593
3594,7593,3594
3595,7594,3595


In [36]:
# Create the tensor the edge_index
train_edge = train[train['Edge'] == 1]
train_edge

,Source,Target,Edge
0,939,3809,1
1,2442,5784,1
2,179,3809,1
3,857,2280,1
4,1358,5722,1
...,...,...,...
10480,147,4217,1
10482,2486,3809,1
10485,2702,5862,1
10490,1240,6164,1


In [37]:
source_new = []
target_new = []
for i in range(train_edge.shape[0]):
    s_new = node_idx.loc[node_idx['old_index'] == np.array(train_edge['Source'])[i], 'new_index'].values[0]
    t_new = node_idx.loc[node_idx['old_index'] == np.array(train_edge['Target'])[i], 'new_index'].values[0]
    source_new.append(s_new)
    target_new.append(t_new)

In [38]:
source_new = torch.from_numpy(np.array(source_new))
target_new = torch.from_numpy(np.array(target_new))
edge_index = torch.stack([source_new, target_new], dim=0)
assert edge_index.size() == (2,5248)

In [39]:
# Create the tensor for node_feature
node_feat = torch.from_numpy(node.values[:,1:]).to(torch.float)
assert node_feat.size() == (3597, 932)

In [40]:
# Create the graph by inserting the data and features directly
G = Data(x=node_feat, edge_index=edge_index)
print(G)    

Data(x=[3597, 932], edge_index=[2, 5248])


# Train-Val Split

In [41]:
split = T.RandomLinkSplit(
    num_val=0.25,
    num_test=0,
    is_undirected=False,
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0,
)
train_data, val_data, _ = split(G)
print('train_data:', train_data)
print('val_data:', val_data)

train_data: Data(x=[3597, 932], edge_index=[2, 3936], edge_label=[7872], edge_label_index=[2, 7872])
val_data: Data(x=[3597, 932], edge_index=[2, 3936], edge_label=[2624], edge_label_index=[2, 2624])


# GNN

In [42]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 256)
        self.conv2 = GCNConv(256, 128)
        self.conv3 = GCNConv(128, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()
        return self.conv4(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=200
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x.to(device), train_data.edge_index.to(device))

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        ).to(device)
        
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0).to(device)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x.to(device), data.edge_index.to(device))
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [44]:
model = Net(G.x.shape[1], 64, 2).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()

model = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=5000)

Epoch: 010, Train Loss: 0.693, Val AUC: 0.523
Epoch: 020, Train Loss: 0.692, Val AUC: 0.586
Epoch: 030, Train Loss: 0.691, Val AUC: 0.606
Epoch: 040, Train Loss: 0.689, Val AUC: 0.597
Epoch: 050, Train Loss: 0.686, Val AUC: 0.581
Epoch: 060, Train Loss: 0.680, Val AUC: 0.561
Epoch: 070, Train Loss: 0.673, Val AUC: 0.548
Epoch: 080, Train Loss: 0.666, Val AUC: 0.536
Epoch: 090, Train Loss: 0.658, Val AUC: 0.536
Epoch: 100, Train Loss: 0.650, Val AUC: 0.538
Epoch: 110, Train Loss: 0.647, Val AUC: 0.539
Epoch: 120, Train Loss: 0.643, Val AUC: 0.535
Epoch: 130, Train Loss: 0.640, Val AUC: 0.538
Epoch: 140, Train Loss: 0.637, Val AUC: 0.539
Epoch: 150, Train Loss: 0.633, Val AUC: 0.540
Epoch: 160, Train Loss: 0.630, Val AUC: 0.537
Epoch: 170, Train Loss: 0.625, Val AUC: 0.532
Epoch: 180, Train Loss: 0.628, Val AUC: 0.531
Epoch: 190, Train Loss: 0.624, Val AUC: 0.527
Epoch: 200, Train Loss: 0.621, Val AUC: 0.527
Epoch: 210, Train Loss: 0.624, Val AUC: 0.525
Epoch: 220, Train Loss: 0.620, Val

# Make Predictions

In [46]:
source_test = []
target_test = []
for i in range(test.shape[0]):
    s_new = node_idx.loc[node_idx['old_index'] == np.array(test['Source'])[i], 'new_index'].values[0]
    t_new = node_idx.loc[node_idx['old_index'] == np.array(test['Target'])[i], 'new_index'].values[0]
    source_test.append(s_new)
    target_test.append(t_new)

source_test = torch.from_numpy(np.array(source_test))
target_test = torch.from_numpy(np.array(target_test))
edge_index_test = torch.stack([source_test, target_test], dim=0)
assert edge_index_test.size() == (2,3498)

In [47]:
# Create the graph by inserting the data and features directly
G_test = Data(x=node_feat, edge_index=edge_index_test, edge_label = torch.zeros(3498), edge_label_index = edge_index_test)
print(G_test)

Data(x=[3597, 932], edge_index=[2, 3498], edge_label=[3498], edge_label_index=[2, 3498])


In [48]:
@torch.no_grad()
def test_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x.to(device), data.edge_index.to(device))
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    median = np.median(out.cpu().numpy())
    preds = [0 if i < median else 1 for i in out.cpu().numpy() ]

    return out, preds

In [49]:
proba, preds = test_link_predictor(model,G_test)

In [50]:
sum(preds)

1749

# Submission

In [51]:
submission = pd.DataFrame()
submission['ID'] = list(range(len(preds)))
submission['Predicted'] = preds
submission.to_csv('submission.csv', index = False)